<a href="https://colab.research.google.com/github/spirosChv/neuro208/blob/main/practicals/Practical_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 4: How inputs inputs interact in time

In this notebook we will see how inputs integrate when they are seperated in time.

In [ ]:
!pip install neuron --quiet

In [ ]:
# @title Download the `.mod` file from github
# @markdown Execute this cell.
 
!rm -rf imbizo2022/
!rm -rf x86_64/
!git clone https://github.com/spirosChv/imbizo2022.git
!mv imbizo2022/mechanisms/vecstim.mod imbizo2022/mechanisms/nmda.mod .
!nrnivmodl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
from neuron.units import mV, ms, um
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

In [ ]:
# Simulation parameters	
tstop = 100 * ms # simulation time (ms)
h.dt = 0.1 * ms  # integration step (ms)
vinit = -65 * mV  # initial voltage (mV)

## Define `h.Sections` and their active/passive parameters

Create a soma and one dendrite, define their anatomical and biophysical properties and connect them.

In [ ]:
soma = h.Section(name='soma')
dend0 = h.Section(name='dend0') 

# Define properties of the soma
soma.diam = 18 * um  # diametr (um)
soma.L = 18 * um  # length (um)
soma.Ra = 100  # axial resistance (Ohm * cm)
soma.cm = 1  # specific membrane capacitance (uF/cm2)   
soma.nseg = 1	 # number of segments

# Insert passive (leak) channels
soma.insert('pas')
for seg in soma:
  soma.e_pas = -65  # leak reversal potential (mV)
  soma.g_pas = 0.0003  # leak maximum conductance (S/cm2)

# Define properties of dend0
dend0.diam = 5 * um  # diametr (um)
dend0.L = 700 * um  # length (um)
dend0.Ra = 100  # axial resistance (Ohm * cm)
dend0.cm = 1  # specific membrane capacitance (uF/cm2)   
dend0.nseg = 11  # number of segments

# Insert passive (leak) channels
dend0.insert('pas')
for seg in dend0:
  seg.e_pas = -65  # leak reversal potential (mV)
  seg.g_pas = 0.0003  # leak maximum conductance (S/cm2)

# Connect the 0 point of dendrite 0 to the zero point of the soma, i.e., soma(0)
dend0.connect(soma(0), 0)

### Task 1: We will add three `h.ExpSyn` in the different parts of the dendrite

- What will happen to the temporal summation of the inputs if they are activated sequentially with 10ms intervals?
- Do you expect the temporal order of activation to affect the output?

In [ ]:
#========== Synaptic stimulation.
# Place three synapses, one in the end, one in the middle and one in the begining of the dendrite.
syn0 = h.ExpSyn(dend0(0.9))
syn0.e = 0  # reversal potential (mV)
syn0.tau = 10  # decay time constant (ms)

syn1 = h.ExpSyn(dend0(0.5))
syn1.e = 0  # reversal potential (mV)
syn1.tau = 10  # decay time constant (ms)

syn2 = h.ExpSyn(dend0(0.1))
syn2.e = 0  # reversal potential (mV)
syn2.tau = 10  # decay time constant (ms)

#========== ...create an artificial spike (an "event" to be delivered to the synapse)...
tsignal = 20
ns = h.NetStim(0.5)
ns.start = tsignal
ns.number = 1

#... and connect the event to the synapses.
wAMPA = 0.003
nc0 = h.NetCon(ns, syn0)
nc0.delay = 20
nc0.weight[0] = wAMPA

nc1 = h.NetCon(ns, syn1)
nc1.delay = 10
nc1.weight[0] = 0.003

nc2 = h.NetCon(ns, syn2)
nc2.delay = 0
nc2.weight[0] = 0.003

In [ ]:
# Calculate and print in terminal the depolarization at the soma and at the middle part of the dend
vsoma_vec = h.Vector().record(soma(0.5)._ref_v) # Membrane potential vector
vdend0_vec = h.Vector().record(dend0(0.5)._ref_v) # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend0_vec, color='red', label='dendrite')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.legend()
plt.show()

print(f'\nSomatic depolarization is {round(vsoma_vec.max()-vsoma_vec[int((tsignal)/h.dt)-1], 2)} mV')

## Task 2: Add NMDA synapses using the `nmda.mod`.

The rise time should be 10 ms and the decay time 75 ms. What do you observe?

In [ ]:
!cat nmda.mod

In [ ]:
#========== Synaptic stimulation.
# Place three synapses, one in the end, one in the middle and one in the begining of the dendrite.
synNMDA0 = ...
synNMDA0.e = 0  # reversal potential (mV)
# TODO: Set the synaptic kinetics

synNMDA1 = ...
synNMDA1.e = 0  # reversal potential (mV)
# TODO: Set the synaptic kinetics

synNMDA2 = ...
synNMDA2.e = 0  # reversal potential (mV)
# TODO: Set the synaptic kinetics

#========== ...create an artificial spike (an "event" to be delivered to the synapse)...
tsignal = 20
ns = h.NetStim(0.5)
ns.start = tsignal
ns.number = 1

#... and connect the event to the synapses.
wNMDA = 0.03
ncNMDA0 = h.NetCon(ns, synNMDA0)
ncNMDA0.delay = 20
ncNMDA0.weight[0] = wNMDA

ncNMDA1 = h.NetCon(ns, synNMDA1)
ncNMDA1.delay = 10
ncNMDA1.weight[0] = wNMDA

ncNMDA2 = h.NetCon(ns, synNMDA2)
ncNMDA2.delay = 0
ncNMDA2.weight[0] = wNMDA

In [ ]:
# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.plot(t_vec, vdend0_vec, color='red', label='dendrite')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')
plt.legend()
plt.show()

print(f'\nSomatic depolarization is {round(vsoma_vec.max()-vsoma_vec[int((tsignal)/h.dt)-1], 2)} mV')

# Homework

Add more sophisticated inputs using the `h.VecStim` object!

**Hint:** To create a VecStim object you need: `spks = h.Vector(spiketimes)`, `vstim = h.VecStim()`, and `vstim.play(spks)`.

In [ ]:
import numpy as np


def poisson_spikes(t1, t2, N, rate=10, dt=0.1):
  """
  Poisson spike generator.
  Parameters
  ----------
  t1 : float
      Simulation time start in miliseconds (ms).
  t2 : float
      Simulation time end in miliseconds (ms).
  N : int, optional
      Number of presynaptic spikes.
  rate : float, optional
      Mean firing rate in Hz. The default is 10.
  dt : float, optional
      Time step in ms. The default is 0.1.
  Returns
  -------
  spks : TYPE
      DESCRIPTION.
  """
  spks = []
  tarray = np.arange(t1, t2+dt, dt)
  for n in range(N):
    spkt = tarray[np.random.rand(len(tarray)) < rate*dt/1000.]  # Determine list of times of spikes
    idx = [n]*len(spkt)  # Create vector for neuron ID number the same length as time
    spkn = np.concatenate([[idx], [spkt]], axis=0).T  # Combine tw lists
    if len(spkn) > 0:
      spks.append(spkn)
  spks = np.concatenate(spks, axis=0)
  return spks

Let's create some Poisson ditsributed spiketimes!

In [ ]:
pre_cells = 10000
spks = poisson_spikes(t1=0, t2=1000, N=pre_cells, rate=25)

plt.figure(figsize=(12, 8))
for i, r in enumerate(np.random.choice(pre_cells, 100, replace=False)):
  spks_i = spks[spks[:, 0] == r][:,1]
  plt.scatter(spks_i, i*np.ones((len(spks_i), )), color='k')
plt.ylabel('neuron id')
plt.xlabel('time (ms)')
plt.show()

In [ ]:
freqs = []
for i in range(pre_cells):
  spks_i = spks[spks[:, 0] == i][:, 1]
  freqs.append(len(spks_i))

plt.figure(figsize=(12, 8))
plt.hist(freqs, bins=20)
plt.ylabel('neuron id')
plt.xlabel('time (ms)')
plt.show()